# Exercise - Hadamard matrix multiplication with SVM

Hadamard matrix multiplication is where the values in matrices **D** and **E** at coordinates (i0,i1) are multiplied together to set the value at coordinates (i0,i1) in matrix **F**.

<figure style="margin-left:auto; margin-right:auto; width:80%;">
    <img style="vertical-align:middle" src="../images/elementwise_multiplication.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Elementwise multiplication of matrices D and E to get F.</figcaption>
</figure>

In this exercise we are going enable the elementwise matrix multiplication code to use fine-grained buffer Shared Virtual Memory (SVM). The source code to edit is located in [mat_elementwise_svm.cpp](mat_elementwise_svm.cpp) and the kernel is in [kernels_elementwise.c](kernels_elementwise.c). Matrices **D** and **E** are read in from disk and matrix **F** is produced as the output. Your task is to tool up this code so that the memory allocation for **matrix F** is using fine-grained buffer SVM.

## The problem

The program to edit is [mat_elementwise_svm.cpp](mat_elementwise_svm.cpp). It runs fine, however matrix F is handled in the usual way. Memory for **F_h** is allocated on the host and **F_d** is allocated on the device. A kernel is run to compute **F_d** from **D_d** and **E_d** and **F_d** is copied back to **F_h** before writing to disk and checking for errors.

In [1]:
!make mat_elementwise_svm.exe; ./mat_elementwise_svm.exe

make: 'mat_elementwise_svm.exe' is up to date.
	               name: gfx1035 
	     Device version: OpenCL 2.0  
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
The output array F_h (as computed with OpenCL) is
----
|  4.50e-01  1.70e-01  2.77e-01  2.21e-02 |
|  2.46e-02  3.48e-02  4.41e-02  2.05e-01 |
|  7.57e-01  4.06e-03  3.90e-01  2.74e-01 |
|  3.16e-01  3.38e-05  9.45e-02  9.03e-01 |
|  1.60e-02  6.24e-03  9.69e-02  4.00e-01 |
|  4.89e-01  4.12e-01  8.46e-01  8.93e-02 |
|  3.23e-01  3.19e-02  2.84e-01  4.18e-01 |
|  2.02e-02  3.38e-01  2.30e-01  1.49e-01 |
----
The CPU solution (F_answer_h) is 
----
|  4.50e-01  1.70e-01  2.77e-01  2.21e-02 |
|  2.46e-02  3.48e-02  4.41e-02  2.05e-01 |
|  7.57e-01  4.06e-03  3.90e-01  2.74e-01 |
|  3.16e-01  3.38e-05  9.45e-02  9.03e-01 |
|  1.60e-02  6.24e-03  9.69e-02  4.00e-01 |
|  4.89e-01  4.12e-01  8.46e-01  8.93e-02 |
|  3.23e-01  3.19e-02  2.84e-01  4.18e-01 |
|  2.

Your task is to make the necessary changes so that the kernel uses **F_svm**, a shared memory allocation on the host for matrix F that uses **fine-grained buffer SVM**. The sollution will eliminate the need for separate handling of **F_d** and **F_h**.

## Tasks

1. Add a check so that the progam will crash out if the runtime is not capable of using CL_DEVICE_SVM_FINE_GRAIN_BUFFER (copy paste this from [mat_mult_fine_buffer_svm.cpp](mat_mult_fine_buffer_svm.cpp)).
1. Allocate memory for **F_svm** using [clSVMAlloc](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clSVMAlloc.html) instead. Don't forget the additional **CL_MEM_SVM_FINE_GRAIN_BUFFER** flag.
1. Remove the code that creates a buffer for **F_d**.
1. Remove all code that allocates and frees memory for **F_h**
1. Replace clSetKernelArg with [clSetKernelArgSVMPointer](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clSetKernelArgSVMPointer.html) for setting argument 2 in the kernel.
1. Remove the read of **F_d** back to the host.
1. Use **F_svm** everywhere instead of **F_h**.
1. Remove the call to clReleaseMemObject for **F_d**.
1. Use [clSVMFree](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clSVMFree.html) to free memory for **F_svm**.

## The answer

You can of course always look at the answer in [mat_elementwise_svm_answer.cpp](mat_elementwise_svm_answer.cpp), but then try to understand how the solution is working.

In [2]:
!make mat_elementwise_svm_answer.exe; ./mat_elementwise_svm_answer.exe -cpu

make: 'mat_elementwise_svm_answer.exe' is up to date.
	               name: AMD Ryzen 7 6800H with Radeon Graphics          
	     Device version: OpenCL 3.0 (Build 0) 
	 global memory size: 32865 MB
	    max buffer size: 16432 MB
	     max local size: (8192,8192,8192)
	     max work-items: 8192
Device supports fine-grained buffer SVM
The output array F_svm (as computed with OpenCL) is
----
|  4.50e-01  1.70e-01  2.77e-01  2.21e-02 |
|  2.46e-02  3.48e-02  4.41e-02  2.05e-01 |
|  7.57e-01  4.06e-03  3.90e-01  2.74e-01 |
|  3.16e-01  3.38e-05  9.45e-02  9.03e-01 |
|  1.60e-02  6.24e-03  9.69e-02  4.00e-01 |
|  4.89e-01  4.12e-01  8.46e-01  8.93e-02 |
|  3.23e-01  3.19e-02  2.84e-01  4.18e-01 |
|  2.02e-02  3.38e-01  2.30e-01  1.49e-01 |
----
The CPU solution (F_answer_h) is 
----
|  4.50e-01  1.70e-01  2.77e-01  2.21e-02 |
|  2.46e-02  3.48e-02  4.41e-02  2.05e-01 |
|  7.57e-01  4.06e-03  3.90e-01  2.74e-01 |
|  3.16e-01  3.38e-05  9.45e-02  9.03e-01 |
|  1.60e-02  6.24e-03  9.69e-02  4

<address>
Written by Dr. Toby Potter of <a href="https://www.pelagos-consulting.com">Pelagos Consulting and Education</a> for the <a href="https://pawsey.org.au">Pawsey Supercomputing Research Centre</a>. All trademarks mentioned are the property of their respective owners.
</address>